In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Define paths
train_path = 'C:/Users/phd/Downloads/datasets (2)/datasets/balance_dataset/train'
test_path = 'C:/Users/phd/Downloads/datasets (2)/datasets/balance_dataset/test'
model_save_path = 'C:/Users/phd/Downloads/datasets (2)/datasets/balance_dataset/fibroid_detection_DCGAN_model.h5'

# Image parameters
img_height, img_width, channels = 150, 150, 3
batch_size = 32

# Prepare the dataset
datagen = ImageDataGenerator(rescale=1./255)
train_dataset = datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Use binary classification
    shuffle=True
)

test_dataset = datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Use binary classification
    shuffle=False
)

# Define the discriminator network as a binary classifier
def build_discriminator(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)  # Binary classifier output
    discriminator = Model(input_layer, x)
    return discriminator

# Build and compile the discriminator (binary classifier)
discriminator = build_discriminator((img_height, img_width, channels))
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the discriminator
epochs = 100
steps_per_epoch = train_dataset.samples // batch_size
discriminator.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)

# Save the trained model
discriminator.save(model_save_path)

# Evaluate the model on the test set
test_loss, test_accuracy = discriminator.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Generate predictions on the test set
test_predictions = discriminator.predict(test_dataset)
test_predicted_classes = np.where(test_predictions > 0.5, 1, 0)
test_true_classes = test_dataset.classes

# Generate classification report
report = classification_report(test_true_classes, test_predicted_classes)
print("Classification Report:")
print(report)


Found 1784 images belonging to 2 classes.
Found 446 images belonging to 2 classes.
Epoch 1/100
55/55 [==============================] - 21s 371ms/step - loss: 0.4391 - accuracy: 0.7871
Epoch 2/100
55/55 [==============================] - 22s 395ms/step - loss: 0.1951 - accuracy: 0.9292
Epoch 3/100
55/55 [==============================] - 23s 411ms/step - loss: 0.1420 - accuracy: 0.9572
Epoch 4/100
55/55 [==============================] - 22s 396ms/step - loss: 0.1213 - accuracy: 0.9629
Epoch 5/100
55/55 [==============================] - 22s 400ms/step - loss: 0.1034 - accuracy: 0.9646
Epoch 6/100
55/55 [==============================] - 22s 397ms/step - loss: 0.0861 - accuracy: 0.9697
Epoch 7/100
55/55 [==============================] - 22s 398ms/step - loss: 0.0763 - accuracy: 0.9737
Epoch 8/100
55/55 [==============================] - 22s 400ms/step - loss: 0.0799 - accuracy: 0.9749
Epoch 9/100
55/55 [==============================] - 20s 355ms/step - loss: 0.0592 - accuracy: 0.9800

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('C:/Users/phd/Downloads/datasets (2)/datasets/balance_dataset/fibroid_detection_DCGAN_model.h5')

# Define image parameters
img_height, img_width = 150, 150

# Define the path to the input image
input_image_path = 'C:\\Users\\phd\\Downloads\\datasets (2)\\datasets\\datasets\\test\\NUF\\1.2.826.0.1.3680043.2.461.11223642.2406300528.jpg'

# Load and preprocess the input image
input_image = image.load_img(input_image_path, target_size=(img_height, img_width))
input_image = image.img_to_array(input_image)
input_image = np.expand_dims(input_image, axis=0)
input_image = input_image / 255.0  # Normalize pixel values

# Predict the class of the input image
prediction = model.predict(input_image)

# Interpret the prediction result
if prediction[0][0] > 0.5:
    print("The input image is classified as having fibroids.")
else:
    print("The input image is classified as not having fibroids.")


C:\Users\phd\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\phd\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


1/1 [==============================] - 0s 127ms/step
The input image is classified as not having fibroids.
